In [ ]:
#import essential packages 

import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize 
from collections import Counter 

In [ ]:
#read data using Pandas as JSON file

data= pd.read_json('data/world_bank_projects.json')

In [ ]:
#check DataFrame to get some initial insight of it 

data.info()
data.head()

In [ ]:
#countries with the most projects -> looking into coloumn:'countryname'

countries = data['countryname']
top_countries = Counter(countries).most_common(10)
print (top_countries)

In [ ]:
#the most common project themes -> we need to load our raw data as json_string in order to use json_normalize 

data_string = json.load((open('data/world_bank_projects.json')))
data_themes = json_normalize(data_string, 'mjtheme_namecode')

#apply Counter on 'code' since not all the projects have 'name'
top_themes = Counter(data_themes['code']).most_common(10)
print (top_themes)

In [ ]:
#Create a table of projects' codes and names without any missing value

#firstreplace '' with Nan and sort DataFrame by code and name
data_themes.replace(r'^\s*$',np.nan,regex=True,inplace=True)
data_themes = data_themes.sort_values(by=['code','name'])

#forward fill the nan's from the rows where you do have a name
data_themes['name'] = data_themes[['name']].fillna(method='ffill')
new_top_themes = Counter(data_themes['name']).most_common(10)
print(new_top_themes)